# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv("WeatherPy_Database.csv")

# Display sample data
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mahebourg,MU,-20.4081,57.7000,68.29,83,75,2.30,broken clouds
1,1,Torbay,CA,47.6666,-52.7314,33.58,77,100,9.22,overcast clouds
2,2,Oktyabrskoye,RU,43.0645,44.7417,28.54,93,100,4.47,overcast clouds
3,3,Bam,IR,29.1060,58.3570,59.83,38,2,3.13,clear sky
4,4,Toumodi,CI,6.5613,-5.0158,69.24,78,32,3.67,scattered clouds


In [3]:
city_data_df.dtypes

Unnamed: 0               int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [4]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip?  75
What is the maximum temperature you would like for your trip?  85


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [5]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)] 
                                    
# Display sample data
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Isangel,VU,-19.5500,169.2667,82.89,64,94,6.80,overcast clouds
8,8,Harper,LR,4.3750,-7.7169,77.67,85,8,7.05,clear sky
14,14,Guanica,PR,17.9716,-66.9080,80.06,73,21,6.98,few clouds
15,15,Cayenne,GF,4.9333,-52.3333,80.64,83,0,2.30,clear sky
20,20,Kapaa,US,22.0752,-159.3190,80.92,62,75,11.50,broken clouds
23,23,Sao Jose Da Coroa Grande,BR,-8.8978,-35.1478,76.93,83,46,6.71,scattered clouds
27,27,Sumbe,AO,-11.2061,13.8437,75.24,75,100,6.42,overcast clouds
28,28,Hilo,US,19.7297,-155.0900,79.25,80,75,5.75,light rain
32,32,Cockburn Town,TC,21.4612,-71.1419,82.13,75,5,17.40,clear sky
34,34,Atuona,PF,-9.8000,-139.0333,78.19,75,2,12.57,clear sky


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [6]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Isangel,VU,-19.5500,169.2667,82.89,64,94,6.80,overcast clouds
8,8,Harper,LR,4.3750,-7.7169,77.67,85,8,7.05,clear sky
14,14,Guanica,PR,17.9716,-66.9080,80.06,73,21,6.98,few clouds
15,15,Cayenne,GF,4.9333,-52.3333,80.64,83,0,2.30,clear sky
20,20,Kapaa,US,22.0752,-159.3190,80.92,62,75,11.50,broken clouds
23,23,Sao Jose Da Coroa Grande,BR,-8.8978,-35.1478,76.93,83,46,6.71,scattered clouds
27,27,Sumbe,AO,-11.2061,13.8437,75.24,75,100,6.42,overcast clouds
28,28,Hilo,US,19.7297,-155.0900,79.25,80,75,5.75,light rain
32,32,Cockburn Town,TC,21.4612,-71.1419,82.13,75,5,17.40,clear sky
34,34,Atuona,PF,-9.8000,-139.0333,78.19,75,2,12.57,clear sky


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [7]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities.copy()

# Display sample data
hotel_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Isangel,VU,-19.5500,169.2667,82.89,64,94,6.80,overcast clouds
8,8,Harper,LR,4.3750,-7.7169,77.67,85,8,7.05,clear sky
14,14,Guanica,PR,17.9716,-66.9080,80.06,73,21,6.98,few clouds
15,15,Cayenne,GF,4.9333,-52.3333,80.64,83,0,2.30,clear sky
20,20,Kapaa,US,22.0752,-159.3190,80.92,62,75,11.50,broken clouds
23,23,Sao Jose Da Coroa Grande,BR,-8.8978,-35.1478,76.93,83,46,6.71,scattered clouds
27,27,Sumbe,AO,-11.2061,13.8437,75.24,75,100,6.42,overcast clouds
28,28,Hilo,US,19.7297,-155.0900,79.25,80,75,5.75,light rain
32,32,Cockburn Town,TC,21.4612,-71.1419,82.13,75,5,17.40,clear sky
34,34,Atuona,PF,-9.8000,-139.0333,78.19,75,2,12.57,clear sky


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [8]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df['Hotel Name'] = np.nan

# Display sample data
hotel_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Hotel Name
5,5,Isangel,VU,-19.5500,169.2667,82.89,64,94,6.80,overcast clouds,NaN
8,8,Harper,LR,4.3750,-7.7169,77.67,85,8,7.05,clear sky,NaN
14,14,Guanica,PR,17.9716,-66.9080,80.06,73,21,6.98,few clouds,NaN
15,15,Cayenne,GF,4.9333,-52.3333,80.64,83,0,2.30,clear sky,NaN
20,20,Kapaa,US,22.0752,-159.3190,80.92,62,75,11.50,broken clouds,NaN
23,23,Sao Jose Da Coroa Grande,BR,-8.8978,-35.1478,76.93,83,46,6.71,scattered clouds,NaN
27,27,Sumbe,AO,-11.2061,13.8437,75.24,75,100,6.42,overcast clouds,NaN
28,28,Hilo,US,19.7297,-155.0900,79.25,80,75,5.75,light rain,NaN
32,32,Cockburn Town,TC,21.4612,-71.1419,82.13,75,5,17.40,clear sky,NaN
34,34,Atuona,PF,-9.8000,-139.0333,78.19,75,2,12.57,clear sky,NaN


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [9]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [10]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# Display sample data
hotel_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Hotel Name
5,5,Isangel,VU,-19.5500,169.2667,82.89,64,94,6.80,overcast clouds,Tanna Lodge
8,8,Harper,LR,4.3750,-7.7169,77.67,85,8,7.05,clear sky,Screensaver
14,14,Guanica,PR,17.9716,-66.9080,80.06,73,21,6.98,few clouds,Copamarina Beach Resort & Spa
15,15,Cayenne,GF,4.9333,-52.3333,80.64,83,0,2.30,clear sky,Hôtel Le Dronmi
20,20,Kapaa,US,22.0752,-159.3190,80.92,62,75,11.50,broken clouds,Sheraton Kauai Coconut Beach Resort
23,23,Sao Jose Da Coroa Grande,BR,-8.8978,-35.1478,76.93,83,46,6.71,scattered clouds,Crôa Mares Hotel
27,27,Sumbe,AO,-11.2061,13.8437,75.24,75,100,6.42,overcast clouds,Hotel Ritz Sumbe
28,28,Hilo,US,19.7297,-155.0900,79.25,80,75,5.75,light rain,Hilo Hawaiian Hotel
32,32,Cockburn Town,TC,21.4612,-71.1419,82.13,75,5,17.40,clear sky,Osprey Beach Hotel
34,34,Atuona,PF,-9.8000,-139.0333,78.19,75,2,12.57,clear sky,Villa Enata


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [12]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Hotel Name
5,5,Isangel,VU,-19.5500,169.2667,82.89,64,94,6.80,overcast clouds,Tanna Lodge
8,8,Harper,LR,4.3750,-7.7169,77.67,85,8,7.05,clear sky,Screensaver
14,14,Guanica,PR,17.9716,-66.9080,80.06,73,21,6.98,few clouds,Copamarina Beach Resort & Spa
15,15,Cayenne,GF,4.9333,-52.3333,80.64,83,0,2.30,clear sky,Hôtel Le Dronmi
20,20,Kapaa,US,22.0752,-159.3190,80.92,62,75,11.50,broken clouds,Sheraton Kauai Coconut Beach Resort
23,23,Sao Jose Da Coroa Grande,BR,-8.8978,-35.1478,76.93,83,46,6.71,scattered clouds,Crôa Mares Hotel
27,27,Sumbe,AO,-11.2061,13.8437,75.24,75,100,6.42,overcast clouds,Hotel Ritz Sumbe
28,28,Hilo,US,19.7297,-155.0900,79.25,80,75,5.75,light rain,Hilo Hawaiian Hotel
32,32,Cockburn Town,TC,21.4612,-71.1419,82.13,75,5,17.40,clear sky,Osprey Beach Hotel
34,34,Atuona,PF,-9.8000,-139.0333,78.19,75,2,12.57,clear sky,Villa Enata


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [ ]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [13]:
# Set the file name.
output_data_file = 'WeatherPy_vacation.csv'

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file)

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [14]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [15]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Tanna Lodge</dd>\n<dt>City</dt><dd>Isangel</dd>\n<dt>Country</dt><dd>VU</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 82.89 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Screensaver</dd>\n<dt>City</dt><dd>Harper</dd>\n<dt>Country</dt><dd>LR</dd>\n<dt>Current Weather</dt><dd>clear sky and 77.67 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Copamarina Beach Resort & Spa</dd>\n<dt>City</dt><dd>Guanica</dd>\n<dt>Country</dt><dd>PR</dd>\n<dt>Current Weather</dt><dd>few clouds and 80.06 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hôtel Le Dronmi</dd>\n<dt>City</dt><dd>Cayenne</dd>\n<dt>Country</dt><dd>GF</dd>\n<dt>Current Weather</dt><dd>clear sky and 80.64 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Sheraton Kauai Coconut Beach Resort</dd>\n<dt>City</dt><dd>Kapaa</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>broken clouds and 80.92 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Crôa Mares Hotel</dd>\n

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [16]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
5,-19.5500,169.2667
8,4.3750,-7.7169
14,17.9716,-66.9080
15,4.9333,-52.3333
20,22.0752,-159.3190
23,-8.8978,-35.1478
27,-11.2061,13.8437
28,19.7297,-155.0900
32,21.4612,-71.1419
34,-9.8000,-139.0333


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [17]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))